# aspects


Aspects provide additional functionality to the classic Object-Oriented pattern.

https://en.wikipedia.org/wiki/Aspect-oriented_programming



## introduction


Types of aspects:

- affect - run after the code
- prefect - run before the code
- inflect - make available part of the affection within the code
- reflect - describe the current affection
- infect - replace part of code with result of another
- effect - return a list of affections the occured or will occur if reference is provided
- expect - deriving from the context find the variables involved and if they do not meet expectation create a "soft assertion"?
- inspect - show debugging information surrounding the specified code


TODO: aspect babel plugin. This will use the transpiler.

TODO: depends on transpiling kernels


## transpiler bindings

Transpile calls that bind functionality to the neessary directives.

TODO: inspect, find calls to aspect and interpret them up front.



### inspect

Logging is one of the most obvious use cases for aspect oriented programming. It makes sense to start with this.  Log messages are inserted at every possible point in the program execution. When transpiled, additional sum information is collected and provided at the end, useful for attaching a unit test and code coverage reporter.



#### the code

inspect every statement?


In [ ]:
var importer = require('../Core')
var {transpile} = importer.import('transpile code')
var {selectAst} = importer.import('select code tree')
var {selectDom} = importer.import('select tree')

var STATEMENTS = `//FunctionDeclaration`

function inspect(ctx, line, filename) {
    console.log(JSON.stringify(ctx), line, filename)
}

function inspectTemplate(ctx, line, filename) {
    // code inserted in to transpiled module
    inspect(ctx, line, filename)
}

function insertInspect(ctx) {
    // TODO: 'replace' transpiler
    var inspect = selectAst(`//FunctionDeclaration`, inspectTemplate)
    var replaceCtx = selectDom(`.//*[./Identifier/@name="ctx"]`, inspect)
    console.log(replaceCtx)
    //ctx.replaceWith(selectDom())
    var parent = selectDom(`parent::*`, ctx)
    parent.insertBefore(inspect, ctx)
}

function transpileInspect(code, filename) {
    return transpile([
        [STATEMENTS, insertInspect]
    ], code)
}

module.exports = {
    inspect,
    inspectTemplate,
    transpileInspect,
}


#### test statement inspector



In [2]:
var importer = require('../Core')
var {transpileInspect} = importer.import('inspect every statement')

var code = `
var escodegen = require('escodegen');
var importer = require('../Core');

function addImports() {

}

`

if(typeof $$ != 'undefined') {
    $$.mime({'text/plain': transpileInspect(code)})
    
    /*
    expected output 
var addImports = importer.import('add missing imports')
var escodegen = require('escodegen');
*/
    
}


compiling /Users/briancullinan/jupyter_ops/Core/aspects.ipynb[0] aka inspect every statement


ReferenceError: selectAst is not defined